In [49]:
import requests
from bs4 import BeautifulSoup
import re
import mysql.connector as SQLC
import pandas as pd
import time

In [2]:
URL = 'https://www.ncbi.nlm.nih.gov/nuccore/NM_003742' #Gene ABCB11

Get da página que contrém o registo

In [12]:
r = requests.get(URL)

Parsing the HTML

In [10]:
soup = BeautifulSoup(r.content, 'html.parser')
soup.find_all('meta')

[<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <meta content="index,nofollow,noarchive" name="robots"/>,
 <meta content="entrez" name="ncbi_app"/>,
 <meta content="nuccore" name="ncbi_db"/>,
 <meta content="genbank" name="ncbi_report"/>,
 <meta content="html" name="ncbi_format"/>,
 <meta content="20" name="ncbi_pagesize"/>,
 <meta content="default" name="ncbi_sortorder"/>,
 <meta content="1" name="ncbi_pageno"/>,
 <meta content="1" name="ncbi_resultcount"/>,
 <meta content="retrieve" name="ncbi_op"/>,
 <meta content="genbank" name="ncbi_pdid"/>,
 <meta content="CE8897445A41EAA1_0138SID" name="ncbi_sessionid"/>,
 <meta content="1387699148" name="ncbi_uidlist"/>,
 <meta content="all" name="ncbi_filter"/>,
 <meta content="false" name="ncbi_stat"/>,
 <meta content="false" name="ncbi_hitstat"/>,
 <meta content="CE8897445A41AE8100000000008A0083.m_32" name="ncbi_phid"/>,
 <meta content="origin-when-cross-origin" name="referrer"/>]

Get do id do gene ABCB11 na base de dados

In [13]:
id = soup.find_all('meta',{'name':'ncbi_uidlist'})[0].attrs['content']

In [21]:
# Procurar um tag meta com um determinado atributo

url = f"https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={id}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000"

r2 = requests.get( url )
print(r2.text)

LOCUS       NM_003742               6934 bp    mRNA    linear   PRI 21-NOV-2023
DEFINITION  Homo sapiens ATP binding cassette subfamily B member 11 (ABCB11),
            mRNA.
ACCESSION   NM_003742
VERSION     NM_003742.4
KEYWORDS    RefSeq; MANE Select.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 6934)
  AUTHORS   Liu H, Irobalieva RN, Kowal J, Ni D, Nosol K, Bang-Sorensen R,
            Lancien L, Stahlberg H, Stieger B and Locher KP.
  TITLE     Structural basis of bile salt extrusion and small-molecule
            inhibition in human BSEP
  JOURNAL   Nat Commun 14 (1), 7296 (2023)
   PUBMED   37949847
  REMARK    GeneRIF: Structural basis of bile salt extrusion and small-molecule
            inhibition in human BSEP.
            Publication Status: Onli

Criamos um ficheiro gb

In [64]:
def save_genbank(text):
    
    match = re.search(r'LOCUS\s+(.*?)\s+\b', text)

    if match:
        nuccore_id = match.group(1)
        print(f'O ID na base de dados Nucleotide é {nuccore_id}.')

        filename = f'{nuccore_id}.gb'
        with open(filename,'w',encoding='utf-8') as gb_file:
            gb_file.write(text)
            print(f'Resultados gravados no ficheiro {filename}.')

save_genbank(r2.text)

O ID na base de dados Nucleotide é NM_003742.
Resultados gravados no ficheiro NM_003742.gb.


In [36]:
def parse_genbank(locus):
    
    i = re.match(r'LOCUS\s+(\w+)', locus)
    if i:
        id = i.group(1)
    organism = ""
    o = re.search(r'SOURCE\s+.+', locus)
    if o:
        s = re.match(r'SOURCE\s+(.+)', o[0] )
        if s:
            organism = s.group(1)
    sequencia = ""
    existe = re.findall(r'^\s+\d+ [actg ]+', locus, re.MULTILINE )
    if existe:
        for linha in existe:
            m = re.match( r'\s+\d+ (.+)', linha, re.DOTALL )
           
            sequencia = sequencia + re.sub(r'\s+', '', m.group(1) )
    return (id, organism, sequencia)

População de bases de dados com os resultados de múltiplas pesquisas

In [54]:
DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="DuarteVelho1",
   password ="123duarte",
    database ="duartetrabalhos"
)

# Criação de um cursor para executar comandos SQL
Cursor = DataBase.cursor()

In [58]:
# Parâmetros para a criação da tabela
TableName ="""CREATE TABLE IF NOT EXISTS genbank
    (
    locusid varchar(255) PRIMARY KEY, 
    dnasource text, 
    dnasequence LONGTEXT
    );
"""

# Executamos os comandos com recurso ao cursor
Cursor.execute(TableName)

In [33]:
tabela = pd.read_sql("SELECT * FROM genbank",DataBase)
tabela

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_5516\1056781267.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT * FROM genbank",DataBase)


,locusid,dnasource,dnasequence


In [37]:
sql = "INSERT INTO genbank (locusid, dnasource, dnasequence) VALUES (%s, %s, %s)"
val = parse_genbank(r2.text)

In [38]:
Cursor.execute(sql, val)

DataBase.commit()  # Executamos as modificações, fazendo commit.

print(Cursor.rowcount, "record inserted.")

1 record inserted.


In [39]:
tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)
tabela

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_5516\2230997112.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)


,locusid,comprimento
0,NM_003742,6934


In [46]:
def get_genbank(gene):
    
    url = f'https://www.ncbi.nlm.nih.gov/nuccore/{gene}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    id = soup.find_all('meta',{'name':'ncbi_uidlist'})[0].attrs['content']
    url = f"https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={id}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000"
    r2 = requests.get(url)
    return r2.text

In [65]:
genes = [
    'NM_153603',
    'NM_001159694',
    'NM_198510'
]

# Verificamos todos os genes utilizando um ciclo for com recurso ao módulo time
#chamamos a função save_genbank que criou os ficheiros gb para o respetivo gene
# para não sobrecarregar o servidor

vals = []
for gene in genes:

    content = get_genbank(gene)
    save_genbank(content)
    vals.append(parse_genbank(content))
    time.sleep(5)

O ID na base de dados Nucleotide é NM_153603.
Resultados gravados no ficheiro NM_153603.gb.
O ID na base de dados Nucleotide é NM_001159694.
Resultados gravados no ficheiro NM_001159694.gb.
O ID na base de dados Nucleotide é NM_198510.
Resultados gravados no ficheiro NM_198510.gb.


Guardar os restantes genes na BD

In [55]:
sql = "INSERT INTO genbank (locusid, dnasource, dnasequence) VALUES (%s, %s, %s)"
for val in vals:
    Cursor.execute(sql, val)
DataBase.commit()

Criar uma tabela para os restantes genes:

In [61]:
tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)
tabela

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_5516\2230997112.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)


,locusid,comprimento
0,NM_001159694,3927
1,NM_003742,6934
2,NM_153603,2935
3,NM_198510,4964


Se quisermos todas as colunas completas da tabela:

In [60]:
tabela = pd.read_sql("SELECT * FROM genbank",DataBase)
tabela

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_5516\1056781267.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT * FROM genbank",DataBase)


,locusid,dnasource,dnasequence
0,NM_001159694,Homo sapiens (human),acattcactacaccttttccatttgctaataaggccctgccaggct...
1,NM_003742,Homo sapiens (human),agaatgatgaaaaccgaggttggaaaaggttgtgaaaccttttaac...
2,NM_153603,Homo sapiens (human),gagggtcattcccagccgcaaactgagtgcgctgagcggggaagcg...
3,NM_198510,Homo sapiens (human),agatcttctggatgtgaacaacagatccaggccagaggtggcatca...


Comentar código

Pôr no git

Fecho do cursos e conexão à base de dados

In [40]:
Cursor.close()
DataBase.close()